<a href="https://colab.research.google.com/github/dbruce1990/backtesting/blob/master/colabs/backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ta
%pip install backtesting

In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
url2 = 'http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 129496 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...
dpkg: error processing package libc-bin (--install):
 installed libc-bin package post-installation script subprocess was interrupted
Errors were encountered while processing:
 libc-bin
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.0/272.0 KB 23.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencie

In [3]:
import os
import pandas as pd
import ta
import talib
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
import concurrent.futures

parent_folder = 'dailyprices_archive'
etfs_folder = os.path.join(parent_folder, "etfs")
stocks_folder = os.path.join(parent_folder, "stocks")


def find_ticker(ticker, startDate=None):
    """Attempts to find the associated stock ticker symbol and return a pandas dataframe.

    Args:
        ticker (str): The stock's ticker symbol. i.e. AAPL, MSFT, TSLA...
        startDate (str, optional): The start date to return the data.
            Defaults to None which returns all available dates. Format YYYY-MM-DD

    Returns:
        DataFrame: A pandas dataframe.
    """
    folder_paths = [etfs_folder, stocks_folder]
    for folder_path in folder_paths:
        file_path = os.path.join(folder_path, ticker.upper() + ".csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col=0, parse_dates=True)
            if startDate != None:
                startDate = pd.to_datetime(startDate)
                df = df.loc[startDate:]
            return df
    return None


def run_backtest(symbol,
                 strategy,
                 startDate=None,
                 cash=10000,
                 commission=0.00,
                 exclusive_orders=True,
                 ):
    """
    startDate: "YYYY-MM-DD"
    """
    df = find_ticker(symbol, startDate)
    if df.empty:
        print(f"\nNo data for {symbol}.\n\n")
        return (symbol, None)

    bt = Backtest(df, strategy=strategy, cash=cash,
                  commission=commission, exclusive_orders=exclusive_orders)
    result = bt.run()
    bt.plot()
    return (symbol, result)


class ChatGPTMACross(Strategy):
    n1 = 10
    n2 = 30

    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)
        df = pd.DataFrame({"high": high, "low": low, "close": close})
        self.atr = self.I(ta.volatility.average_true_range,
                          high=df["high"], low=df["low"], close=df["close"])
        self.stop_loss_level = 0.9

    def next(self):
        if self.sma1[-1] > self.sma2[-1]:
            self.buy()
        elif self.sma1[-1] < self.sma2[-1]:
            self.sell()

        self.update_trailing_stop()

    def update_trailing_stop(self):
        for trade in self.trades:
            # print("Trade:", trade)
            # print("Trade entry price:", trade.entry_price)
            # print("ATR:", self.atr[-1])
            trade.sl = trade.entry_price - self.atr[-1] * self.stop_loss_level
            # print("Trade stop loss:", trade.sl)


ModuleNotFoundError: ignored

In [ ]:
class SMAcross(Strategy):

    n1 = 9
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
class ThinkScriptStrategy(Strategy):
    n = 10
    movingAvgLength = 200
    MACDFastLength = 12
    MACDSlowLength = 26
    MACDLength = 9
    period_sma = 50
    period_bb = 20
    mult_bb = 2.0
    ShowMA = True

    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        self.mvAvg = self.I(ta.trend.sma_indicator,
                            pd.Series(close), self.movingAvgLength)
        self.macdLine = self.I(ta.trend.macd, pd.Series(
            close), window_fast=self.MACDFastLength, window_slow=self.MACDSlowLength)
        self.sma = self.I(ta.trend.sma_indicator,
                          pd.Series(close), self.period_sma)
        self.upperBB = self.I(ta.volatility.bollinger_hband, pd.Series(
            close), window=self.period_bb, window_dev=self.mult_bb)
        self.lowerBB = self.I(ta.volatility.bollinger_lband, pd.Series(
            close), window=self.period_bb, window_dev=self.mult_bb)
        self.rsi = self.I(talib.RSI, close)
        self.stoch = self.I(talib.STOCH, high, low, close)

    def next(self):
        sentiment_sma = 1 if self.data.Close[-1] > self.sma[-1] else -1
        sentiment_bb = 1 if self.data.Close[-1] > self.upperBB[-1] else -1
        sentiment_rsi = 1 if self.rsi[-1] > 50 else -1
        sentiment_stoch = 1 if self.stoch[-1] > 50 else -1
        sentiment_diffZero = 1 if self.macdLine[-1] > 0 else -1
        sentiment_closeMA = 1 if self.data.Close[-1] > self.mvAvg[-1] else -1
        sentiment = sentiment_sma + sentiment_diffZero + sentiment_closeMA + sentiment_rsi + sentiment_stoch

        if sentiment > 0 and crossover(self.macdLine, 0):
            self.buy()
        elif sentiment < 0 or crossover(0, self.macdLine):
            self.sell()

In [25]:
a, adf = run_backtest("AAPL", ThinkScriptStrategy, "2018-01-01")
print(a, adf)

AAPL Start                     2018-01-02 00:00:00
End                       2020-04-01 00:00:00
Duration                    820 days 00:00:00
Exposure Time [%]                   64.310954
Equity Final [$]                 22594.010849
Equity Peak [$]                  24448.171692
Return [%]                         125.940108
Buy & Hold Return [%]               39.852555
Return (Ann.) [%]                   43.750161
Volatility (Ann.) [%]               45.393143
Sharpe Ratio                         0.963806
Sortino Ratio                        2.177857
Calmar Ratio                         2.271259
Max. Drawdown [%]                  -19.262514
Avg. Drawdown [%]                   -4.157866
Max. Drawdown Duration      175 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                  118
Win Rate [%]                        52.542373
Best Trade [%]                      48.134479
Worst Trade [%]                     -9.725565
Avg. Trade [%]               

In [ ]:
am, amdf = run_backtest("AMZN", ThinkScriptStrategy, "2018-01-01")
print(am, amdf)


AMZN Start                     2018-01-02 00:00:00
End                       2020-04-01 00:00:00
Duration                    820 days 00:00:00
Exposure Time [%]                   64.487633
Equity Final [$]                  7527.659912
Equity Peak [$]                   12379.17041
Return [%]                         -24.723401
Buy & Hold Return [%]               60.444398
Return (Ann.) [%]                  -11.877793
Volatility (Ann.) [%]               23.604024
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -40.191954
Avg. Drawdown [%]                  -15.222445
Max. Drawdown Duration      464 days 00:00:00
Avg. Drawdown Duration      105 days 00:00:00
# Trades                                  164
Win Rate [%]                        45.731707
Best Trade [%]                      10.472287
Worst Trade [%]                    -12.900033
Avg. Trade [%]               

In [21]:
n, ndf = run_backtest("NVDA", ThinkScriptStrategy, "2018-01-01")
print(n, ndf)

NVDA Start                     2018-01-02 00:00:00
End                       2020-04-01 00:00:00
Duration                    820 days 00:00:00
Exposure Time [%]                   64.487633
Equity Final [$]                  4082.240395
Equity Peak [$]                  15935.770042
Return [%]                         -59.177596
Buy & Hold Return [%]               21.931277
Return (Ann.) [%]                  -32.894132
Volatility (Ann.) [%]               98.181463
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -95.69365
Avg. Drawdown [%]                  -27.443641
Max. Drawdown Duration      464 days 00:00:00
Avg. Drawdown Duration      104 days 00:00:00
# Trades                                  142
Win Rate [%]                             50.0
Best Trade [%]                        17.1395
Worst Trade [%]                    -36.725297
Avg. Trade [%]               

In [24]:
f, fdf = run_backtest("FB", ThinkScriptStrategy, "2018-01-01")
print(f, fdf)

FB Start                     2018-01-02 00:00:00
End                       2020-04-01 00:00:00
Duration                    820 days 00:00:00
Exposure Time [%]                   64.487633
Equity Final [$]                 10085.758759
Equity Peak [$]                   12595.99025
Return [%]                           0.857588
Buy & Hold Return [%]              -12.027336
Return (Ann.) [%]                    0.380919
Volatility (Ann.) [%]               31.790427
Sharpe Ratio                         0.011982
Sortino Ratio                        0.018927
Calmar Ratio                         0.011131
Max. Drawdown [%]                  -34.222805
Avg. Drawdown [%]                   -9.901012
Max. Drawdown Duration      385 days 00:00:00
Avg. Drawdown Duration       65 days 00:00:00
# Trades                                  143
Win Rate [%]                        51.048951
Best Trade [%]                      12.818726
Worst Trade [%]                    -10.761753
Avg. Trade [%]                 

In [23]:
t, tdf = run_backtest("TSLA", ThinkScriptStrategy, "2018-01-01")
print(t, tdf)

TSLA Start                     2018-01-02 00:00:00
End                       2020-04-01 00:00:00
Duration                    820 days 00:00:00
Exposure Time [%]                   54.063604
Equity Final [$]                          0.0
Equity Peak [$]                  10343.730316
Return [%]                             -100.0
Buy & Hold Return [%]               50.238667
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]              145.647266
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                      -100.0
Max. Drawdown Duration      530 days 00:00:00
Avg. Drawdown Duration      530 days 00:00:00
# Trades                                  159
Win Rate [%]                         45.91195
Best Trade [%]                       9.381569
Worst Trade [%]                   -104.116619
Avg. Trade [%]               

In [1]:
class DCA(Strategy):
  dow = 0
  def init(self):
    self.day_of_week = self.I(lambda x: x, self.data.Close.s.index.dayofweek)
    print(self.dow)

  def next(self):
    if self.day_of_week == self.dow:
      self.buy(size=1)

goog_df = find_ticker("GOOG", "2015-01-01")
bt = Backtest(goog_df, DCA, cash=50000)
# stats = bt.run()
stats = bt.optimize(dow=range(0, 5, 1))
print(stats)
bt.plot()


NameError: ignored